In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import KMeans

SEED=90089

In [3]:
# Read Data
patient_df = pd.read_csv('../data/hyperglycemic_patients.csv')

#Print shape and data preview
print("Dataframe shape:   ", patient_df.shape)
print("Dataframe Features:", patient_df.columns)
patient_df.head()

Dataframe shape:    (12872, 17)
Dataframe Features: Index(['subject_id', 'stay_id', 'glucose_max', 'anchor_age', 'dod', 'gender',
       'dbp_mean', 'sbp_mean', 'glucose_mean', 'heart_rate_mean', 'spo2_mean',
       'resp_rate_mean', 'temperature_mean', 'apsiii', 'glucose_score', 'los',
       'avg_bmi_value'],
      dtype='object')


,subject_id,stay_id,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,los,avg_bmi_value
0,12041046,31249096,277.0,36,0,0,64.320000,112.880000,221.272727,101.800000,92.000000,17.346154,37.186250,38,3,1.938414,26.750000
1,17421995,38100564,2340.0,35,0,0,77.769231,122.961538,468.592593,105.361111,96.093023,19.813953,36.727586,87,5,4.472905,30.558477
2,10352416,35043893,531.0,23,1,0,64.169118,119.661765,275.352941,154.028986,83.698413,25.584615,39.725000,158,5,11.636042,30.558477
3,16924291,34261137,259.0,19,1,1,49.300000,74.200000,259.000000,89.750000,70.000000,26.333333,31.900000,88,3,0.309664,30.558477
4,13553087,36461140,398.0,31,0,0,78.600000,133.800000,285.000000,100.826087,90.304348,25.173913,36.593333,34,5,2.794167,30.558477


### Assign labels to `los` based on `>3Days` and `>7days` inspired based-on prev. research:
https://arxiv.org/pdf/1907.08322v2.pdf

`los` data was very right-skewed from the `exploratory_data_analysis.ipynb` from `visualizations`. 

In [48]:
labelled_patient_df = patient_df.copy()
label_lst = []
for idx, row in labelled_patient_df.iterrows():
    if row['los'] >= 7: 
        label_lst.append(2)
    elif row['los'] > 3:
        label_lst.append(1)
    else:
        label_lst.append(0)


labelled_patient_df['label'] = pd.Series(label_lst)

In [49]:
labelled_patient_df.label.value_counts()

0    8389
1    3165
2    1318
Name: label, dtype: int64

In [1]:
# pd.qcut(np.array(patient_df['los']), q=3, labels=['short', 'medium', 'anomaly']).value_counts()

In [45]:
# Export label data to csv
labelled_patient_df.to_csv('../data/hyperglycemic_patients_w_bin_categories.csv')

In [78]:
# # Data split
# X = label_df.loc[:, label_df.columns != 'label']
# y = label_df.loc[:, label_df.columns == 'label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=89)
# X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=89)